# You Work at the North Pole
Every day, little children around the world write letters to Santa telling him what they want for christmas. Santa reads these letters, and if the child is nice, the gift gets made by one of the elves, wrapped, and put in santa's sleigh to be delivered on Christmas Eve. If the child is naughty, Santa puts coal in their stocking. Coal doesn't need to be wrapped, but it does need to be put in Santa's sleigh.

In an attempt to teach myself threading in python, I'm going to create a working, multi-threaded model of Santa's workshop. 

## All By My Elf
For now, let's assume Santa's workshop has undergone some budget cuts, and only has one elf employed. Making a present takes a variable amount of time--let's say between 0 (maybe the child wished for no presents?) and anything less than 1 hour. Wrapping a present takes a constant amount of time: 10 minutes. For now, we're just going to assume all children are good, so these two tasks can be represented by the following functions:

In [1]:
import time
import random

def make_wish_come_true(wish):
    time.sleep(random.random())
    return wish
        
def wrap_present(present):
    time.sleep(0.1)
    return present

So to fill Santa's sleigh, our lone elf gathers the letters, makes the toys, wraps the toys, and puts them in the sleigh. If a child is naughty, we'll just toss some coal directly into the sleigh; no use wasting an elf's precious time on a bratty kid.

In [2]:
import queue

def fill_santas_sleigh(childrens_letters, santas_list):
    # Santa gives nice kids' letters to elves so they can make them into toys.
    letters = []
    # Finished toys sit in a pile waiting to be wrapped.
    toys = []
    # Finished, wrapped toys.
    sleigh = []
    
    for child, wish in childrens_letters.items():
        if santas_list[child] == "Naughty":
            sleigh.append("Coal for " + child)
        else:
            letters.append(wish + " for " + child)
            
    while letters:
        toy = make_wish_come_true(letters.pop())
        toys.append(toy)

    while toys:
        present = wrap_present(toys.pop())
        sleigh.append(present)
    
    return sleigh

To make sure this all works the way we expect it to, here's some sample data and a test to run it through!

In [3]:
childrens_letters_2017 = {
    "Jimmy":"Racecar",
    "Alison":"Nerf gun",
    "Steve":"Bicycle",
    "Gwendylyn":"Skateboard",
    "Lewis":"Action figure",
    "Kate":"Nintendo",
    "Alex":"Coloring book",
    "Sue":"Stuffed animal"
}

santas_list_2017 = {
    "Jimmy":"Nice",
    "Alison":"Nice",
    "Steve":"Naughty",
    "Gwendylyn":"Nice",
    "Lewis":"Naughty",
    "Kate":"Naughty",
    "Alex":"Nice",
    "Sue":"Nice"
}

sleigh_2017 = fill_santas_sleigh(childrens_letters_2017, santas_list_2017)

print (sleigh_2017)

['Coal for Steve', 'Coal for Lewis', 'Coal for Kate', 'Racecar for Jimmy', 'Nerf gun for Alison', 'Skateboard for Gwendylyn', 'Coloring book for Alex', 'Stuffed animal for Sue']


## Thread The Halls
Ok, so now the workshop is back in the black, and we now have a bunch of elves, not just one. But our `fill_santas_sleigh` function still operates the same, and the new recruits are getting bored. Wouldn't it be great if multiple elves could make and wrap presents in parallel? 

This is where threading comes in. Each elf is a thread, and it can either make or wrap a gift. So if we have "n" elves, we would want each elf to operate like this:

1. If there are toys lying around waiting to be wrapped, wrap them and put them in the sleigh.
2. If there aren't any toys made, go get a letter from santa and make that wish come true.
3. If everything's made, wrapped, and in the sleigh, *exit* the workshop.

There's an important caveat to realize here: if there are multiple elves looking for something to do, they need to do different things. For example, we don't want our little workers bickering over letters or breaking toys because they're both trying to wrap the same one. The solution? Locks. 

We want to make sure only one elf talks to Santa at a time, so if Santa is already busy giving a letter to another elf, other elves should quietly wait their turn. We'll call this a `letter_lock`. Similarly, to avoid fights with elves wanting to wrap the same toy (or elves getting hit by toys being thrown onto the pile), only one elf should approach the pile of toys at a time. The other elves *politely waiting their turn*--meaning we'll also have a `toy_lock`. Finally, we don't want elves throwing toys into the sleigh at the same time--they could collide and break! Christmas would be ruined! To avoid this catastrophe, we'll have a `sleigh_lock`.

> ### A quick note on locks...
> Locks are really important if the resource you're trying to access isn't threadsafe. Lists are a good example of this, but it's worth noting that there are other classes (queues come to mind) that are indeed threadsafe, so you don't have to bother with locks. Therefore, I'm intentionally choosing a sub-optimal data structure to more completely illustrate how multithreading works.

\*cough cough\* With that non-metaphor-fitting blabber out of the way...let's create our own Elf class that is a subclass of Thread (found in the threading python module). During each elf's orientation training (`__init__` for all you programmers), he'll be given a badge with his name, get shown where to find Santa (who has the children's letters), the pile of toys, and Santa's sleigh. He'll also be told about the one-elf-at-a-time policies for each.

In [4]:
import threading

class Elf (threading.Thread):
    def __init__(self, name, letter_location, toy_location, sleigh_location, letter_policy, toy_policy, sleigh_policy):
        threading.Thread.__init__(self)
        self.name = name
        self.letters = letter_location
        self.toys = toy_location
        self.sleigh = sleigh_location
        self.turn_for_letter = letter_policy
        self.turn_for_toy = toy_policy
        self.turn_for_sleigh = sleigh_policy
        print (self.name, "is ready to make some toys!")
    
    def run(self):
        while self.toys or self.letters:
            # Grab a toy and let the next elf take their turn.
            toy = None
            with self.turn_for_toy:
                if self.toys:
                    toy = self.toys.pop()
            # Wrap that toy and put it in the sleigh.
            if toy:
                present = wrap_present(toy)
                with self.turn_for_sleigh:
                    self.sleigh.append(present)
                print (self.name, "wrapped a", present, "and threw it in the sleigh")
                time.sleep(random.random())
                continue

            # Grab a letter and let the next elf take their turn.
            wish = None
            with self.turn_for_letter:
                if self.letters:
                    wish = self.letters.pop()
            # Make a toy and put it in the pile.
            if wish:
                toy = make_wish_come_true(wish)
                with self.turn_for_toy:
                    self.toys.append(toy)
                print (self.name, "made a", wish, "and put it in the toy pile")
                time.sleep(random.random())
                continue
                
        print (self.name, "is punching out for the day!")

You'll notice in the code above that our elf class has a `run` method that follows the 3 steps we outlined above for how an elf should behave. When the function finishes, the elf (thread) exits. Notice this also holds a lot of the code we'd previously placed in `fill_santas_sleigh`, which means we'll want to modify that code as well. The multi-threaded code looks like this:

In [5]:
santa_lock = threading.Lock()
toy_lock = threading.Lock()
sleigh_lock = threading.Lock()

def everyone_fill_santas_sleigh(childrens_letters, santas_list, elves_names):
    # Santa gives nice kids' letters to elves so they can make them into toys.
    letters = []
    # Finished toys sit in a pile waiting to be wrapped.
    toys = []
    # Finished, wrapped toys.
    sleigh = []
    
    for child, wish in childrens_letters.items():
        if santas_list[child] == "Naughty":
            sleigh.append("Coal for " + child)
        else:
            letters.append(wish + " for " + child)
    
    elves = []
    for name in elves_names:
        elf = Elf(name, letters, toys, sleigh, santa_lock, toy_lock, sleigh_lock)
        elf.start()
        elves.append(elf)

    # Wait for all the elves to finish working before closing the workshop for the day.
    for elf in elves:
        elf.join()
    
    time.sleep(0.1)
    print ("All clear! Shutting down the workshop for the day.")
    return sleigh

# It's A Christmas Miracle!
That's everything required for a multi-threaded version of the North Pole. To make sure everything's behaving as expected, here's our previous example, this time with the addition of 3 elves.

In [6]:
letters_2017 = {
    "Jimmy":"Racecar",
    "Alison":"Nerf gun",
    "Steve":"Bicycle",
    "Gwendylyn":"Skateboard",
    "Lewis":"Action figure",
    "Kate":"Nintendo",
    "Alex":"Coloring book",
    "Sue":"Stuffed animal"
}

santas_list_2017 = {
    "Jimmy":"Nice",
    "Alison":"Nice",
    "Steve":"Naughty",
    "Gwendylyn":"Nice",
    "Lewis":"Naughty",
    "Kate":"Naughty",
    "Alex":"Nice",
    "Sue":"Nice"
}

elves_names_2017 = {
    "Flint",
    "Peabody",
    "Keebler"
}

sleigh_2017 = everyone_fill_santas_sleigh(letters_2017, santas_list_2017, elves_names_2017)

print (sleigh_2017)

Flint is ready to make some toys!
Peabody is ready to make some toys!
Keebler is ready to make some toys!
Flint made a Stuffed animal for Sue and put it in the toy pile
Keebler made a Skateboard for Gwendylyn and put it in the toy pile
Peabody made a Coloring book for Alex and put it in the toy pile
Flint wrapped a Coloring book for Alex and threw it in the sleigh
Keebler wrapped a Skateboard for Gwendylyn and threw it in the sleigh
Peabody wrapped a Stuffed animal for Sue and threw it in the sleigh
Keebler made a Nerf gun for Alison and put it in the toy pile
Flint made a Racecar for Jimmy and put it in the toy pile
Keebler wrapped a Nerf gun for Alison and threw it in the sleigh
Peabody wrapped a Racecar for Jimmy and threw it in the sleigh
Flint is punching out for the day!
Keebler is punching out for the day!
Peabody is punching out for the day!
All clear! Shutting down the workshop for the day.
['Coal for Steve', 'Coal for Lewis', 'Coal for Kate', 'Coloring book for Alex', 'Skateb

In [15]:
from urllib.request import urlopen
word_site = "http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
response = urlopen(word_site)
txt = response.read()
words = txt.splitlines()

all_the_elves = list(map(str, range(100)))
all_the_letters = {}
naughty_or_nice = {}
for i in range(len(words)):
    name = "child" + str(i)
    all_the_letters[name] = str(words[i])
    is_naughty = random.choice([True, False])
    naughty_or_nice[name] = "Naughty" if is_naughty else "Nice"

big_ass_sleigh = everyone_fill_santas_sleigh(all_the_letters, naughty_or_nice, all_the_elves)
print (big_ass_sleigh)

0 is ready to make some toys!
1 is ready to make some toys!
2 is ready to make some toys!
3 is ready to make some toys!
4 is ready to make some toys!
5 is ready to make some toys!
6 is ready to make some toys!
7 is ready to make some toys!
8 is ready to make some toys!
9 is ready to make some toys!
10 is ready to make some toys!
11 is ready to make some toys!
12 is ready to make some toys!
13 is ready to make some toys!
14 is ready to make some toys!
15 is ready to make some toys!
16 is ready to make some toys!
17 is ready to make some toys!
18 is ready to make some toys!
19 is ready to make some toys!
20 is ready to make some toys!
21 is ready to make some toys!
22 is ready to make some toys!
23 is ready to make some toys!
24 is ready to make some toys!
25 is ready to make some toys!
26 is ready to make some toys!
27 is ready to make some toys!
28 is ready to make some toys!
29 is ready to make some toys!
30 is ready to make some toys!
31 is ready to make some toys!
32 is ready to mak

455750   made amade awrapped a   b"you've" for child25406b'youngish' for child25397b'yin' for child25369   and put it in the toy pileand put it in the toy pileand threw it in the sleigh


21 made a b'zigging' for child25453 and put it in the toy pile
77 wrapped a b'yourselves' for child25403 and threw it in the sleigh
36 wrapped a b'zounds' for child25481 and threw it in the sleigh
73 wrapped a b'yeasty' for child25349 and threw it in the sleigh
11 6117made a   wrapped amade ab'zombie' for child25472   b'yaw' for child25338b'Zimmerman' for child25458and put it in the toy pile  
and threw it in the sleighand put it in the toy pile

62 made a82  wrapped ab'yond' for child25385  b'zooplankton' for child25477and put it in the toy pile 
and threw it in the sleigh
8516 wrapped a wrapped a  b'yam' for child25320b'yoke' for child25379  and threw it in the sleighand threw it in the sleigh

68 made a b'yipping' for child25371 and put it in the toy pile
53 4made a  made ab"you're" for child25401 

11 made a b'workout' for child25212 and put it in the toy pile
55 made a b'Wyoming' for child25296 and put it in the toy pile
90564   wrapped awrapped amade a   b'Wuhan' for child25286b'workpiece' for child25213b'worse' for child25227   and threw it in the sleighand threw it in the sleighand put it in the toy pile


82 made a b'worsen' for child25228 and put it in the toy pile
49 made a b'wrist' for child25267 and put it in the toy pile
81 wrapped a b'workmen' for child25211 and threw it in the sleigh
33 wrapped a b'workstation' for child25218 and threw it in the sleigh
71 75made a  wrapped ab'wren' for child25257  b'worn' for child25224and put it in the toy pile 
and threw it in the sleigh
30 wrapped a b"x's" for child25307 and threw it in the sleigh
88 made a b'wreck' for child25255 and put it in the toy pile
34 made a b'wristwatch' for child25269 and put it in the toy pile
10 wrapped a b'workout' for child25212 and threw it in the sleigh
24 wrapped a b'Wyoming' for child25296 and th

1 wrapped a b'woodshed' for child25184 and threw it in the sleigh
352  made awrapped a  b'woodchuck' for child25172b'Winston' for child25091  and put it in the toy pileand threw it in the sleigh

3015  wrapped amade a  b'woodland' for child25178b'Winthrop' for child25095  and threw it in the sleighand put it in the toy pile

44 made a b'wombat' for child25157 and put it in the toy pile
86 wrapped a b'wipe' for child25098 and threw it in the sleigh
21 made a b'wolf' for child25148 and put it in the toy pile
82 26wrapped a  made ab'windup' for child25063  b'wood' for child25168and threw it in the sleigh 
and put it in the toy pile
66 made a b'windstorm' for child25060 and put it in the toy pile
96 made a b'Wolcott' for child25146 and put it in the toy pile
45 made a b'Windsor' for child25059 and put it in the toy pile
3911  wrapped amade a  b'woodside' for child25185b'wold' for child25147  and threw it in the sleighand put it in the toy pile

75 wrapped a b'wireman' for child25100 and th

42 made a b'white' for child24941 and put it in the toy pile
1782  wrapped awrapped a  b'Williamson' for child25033b'Winchester' for child25048  and threw it in the sleighand threw it in the sleigh

262 made a  made ab'why' for child24982  b'widen' for child24988and put it in the toy pile 
and put it in the toy pile
58 made a b'Wilhelmina' for child25022 and put it in the toy pile
89 wrapped a b'wildcat' for child25010 and threw it in the sleigh
1 wrapped a b'whiten' for child24946 and threw it in the sleigh
4174 made a wrapped a b'widthwise' for child24994  and put it in the toy pileb'Williamsburg' for child25032
 and threw it in the sleigh
62 wrapped a b'whitewash' for child24948 and threw it in the sleigh
2867  made awrapped a  b'widow' for child24992 b'wholehearted' for child24965and threw it in the sleigh 
and put it in the toy pile
23 wrapped a b'white' for child24941 and threw it in the sleigh
81 made a b'wick' for child24985 and put it in the toy pile
86 wrapped a b'why' for ch

...

69 wrapped a b'Albania' for child563 and threw it in the sleigh
24 made a b'alderman' for child590 and put it in the toy pile
72 wrapped a b'airmass' for child527 and threw it in the sleigh
1 made a b'airlift' for child522 and put it in the toy pile
90 wrapped a b'alchemy' for child579 and threw it in the sleigh
3 made a b'aide' for child501 and put it in the toy pile
92 made a b'Aiken' for child503 and put it in the toy pile
14 wrapped a b'airstrip' for child533 and threw it in the sleigh
44 wrapped a b'Alameda' for child553 and threw it in the sleigh
23 made a b'Albert' for child568 and put it in the toy pile
6217 wrapped a b'Alasdair' for child557  wrapped aand threw it in the sleigh 
b'algaecide' for child615 and threw it in the sleigh
73 wrapped a b'airmail' for child525 and threw it in the sleigh
85 made a b'Alcoa' for child581 and put it in the toy pile
20 wrapped a b'aldermen' for child591 and threw it in the sleigh
918438   wrapped awrapped a  b"ain't" for child509wrapped ab'A

32 wrapped a b'advocacy' for child374 and threw it in the sleigh
39 made a b'affront' for child418 and put it in the toy pile
68 made a b'affair' for child399 and put it in the toy pile
5 wrapped a b'affirmative' for child410 and threw it in the sleigh
98 made a b'affirmation' for child409 and put it in the toy pile
76 made a b'afloat' for child425 and put it in the toy pile
17 wrapped a b'advent' for child358 and threw it in the sleigh
0 made a b'advance' for child355 and put it in the toy pile
84 made a b'Afrikaner' for child435 and put it in the toy pile
45 made a b'affluent' for child413 and put it in the toy pile
55 wrapped a b'advisable' for child369 and threw it in the sleigh
24 made a b'aerie' for child384 and put it in the toy pile
2259 wrapped a  wrapped ab'Afrikaans' for child434  b'aftermath' for child443and threw it in the sleigh 
and threw it in the sleigh
54 made a b'Adriatic' for child342 and put it in the toy pile
47 made a b'afford' for child414 and put it in the toy 

91 wrapped a b'adenine' for child279 and threw it in the sleigh
85 made a b'adherent' for child286 and put it in the toy pile
60 38made a  wrapped ab'acquit' for child213  b'adequate' for child284and put it in the toy pile 
and threw it in the sleigh
66 wrapped a b'adjunct' for child301 and threw it in the sleigh
98 wrapped a b'acrophobia' for child225 and threw it in the sleigh
93 wrapped a b'acrid' for child218 and threw it in the sleigh
58 made a b'across' for child228 and put it in the toy pile
0 wrapped a b'admission' for child316 and threw it in the sleigh
59 made a17  b'actress' for child241wrapped a  and put it in the toy pileb'admiration' for child313
 and threw it in the sleigh
90 made a b'adhesive' for child288 and put it in the toy pile
52 made a b'Achaean' for child186 and put it in the toy pile
12 made a b'acquitting' for child215 and put it in the toy pile
68 wrapped a b'acquit' for child213 and threw it in the sleigh
187778  made a made a b'Adamson' for child257made a  

8877809714    6 wrapped amade awrapped amade a made a    wrapped a b'accidental' for child144b'absorptive' for child101b'acclamation' for child147b'abound' for child66 b'accusation' for child173    b'account' for child160 and threw it in the sleighand put it in the toy pileand threw it in the sleighand put it in the toy pile and put it in the toy pile



and threw it in the sleigh

79 wrapped a b'accrual' for child167 and threw it in the sleigh
65 wrapped a b'absurd' for child108 and threw it in the sleigh
74 wrapped a b'accompanist' for child152 and threw it in the sleigh
23 made a b'abominable' for child60 19and put it in the toy pile 
made a b'accident' for child143 and put it in the toy pile
1 made a b'accredit' for child164 and put it in the toy pile
92 made a b'able' for child50 and put it in the toy pile
61 made a b'accompany' for child153 and put it in the toy pile
71 wrapped a b'Abram' for child74 and threw it in the sleigh
43 wrapped a b'abound' for child66 and threw it in th

16 is punching out for the day!
9613 is punching out for the day!
 made a b'abash' for child13 and put it in the toy pile
50 is punching out for the day!
82 is punching out for the day!
48 is punching out for the day!
9462  is punching out for the day!is punching out for the day!

9951 is punching out for the day!
96 is punching out for the day! 
is punching out for the day!
18 made a b'abate' for child14 and put it in the toy pile
59 is punching out for the day!
71 wrapped a b'abash' for child13 and threw it in the sleigh
49 is punching out for the day!
71 is punching out for the day!
91 is punching out for the day!
26 is punching out for the day!
66 is punching out for the day!
33 is punching out for the day!
39 wrapped a b'abate' for child14 and threw it in the sleigh
65 is punching out for the day!
3921  is punching out for the day!is punching out for the day!

3 is punching out for the day!
52 is punching out for the day!
18 is punching out for the day!
25 is punching out for the 